In [1]:
from scraper import Scraper as BoxscoreScraper

In [2]:
scraper = BoxscoreScraper(year=2022)

In [ ]:
scraper.get_season_boxscores()

Beginning scraping for 2022-2023 season



  0%|          | 0/18 [00:00<?, ?it/s]

Scraping boxscores for Week 1
Succesfully scraped boxscores for Week 1

Scraping boxscores for Week 2
Succesfully scraped boxscores for Week 2

Scraping boxscores for Week 3
Succesfully scraped boxscores for Week 3

Scraping boxscores for Week 4
Succesfully scraped boxscores for Week 4

Scraping boxscores for Week 5
Succesfully scraped boxscores for Week 5

Scraping boxscores for Week 6
Succesfully scraped boxscores for Week 6

Scraping boxscores for Week 7
Succesfully scraped boxscores for Week 7

Scraping boxscores for Week 8
Succesfully scraped boxscores for Week 8

Scraping boxscores for Week 9
Succesfully scraped boxscores for Week 9

Scraping boxscores for Week 10
Succesfully scraped boxscores for Week 10

Scraping boxscores for Week 11
Succesfully scraped boxscores for Week 11

Scraping boxscores for Week 12
Succesfully scraped boxscores for Week 12

Scraping boxscores for Week 13


In [4]:
from filing import Filing

import pandas as pd
class Cleaning:

    def __init__(self, **kwargs):
        """
        This class will contain the functionality to combine, clean, wrangle, and partition all the boxscores
        """

        self.year: int = int(kwargs.get('year', 2023))
        self.season: str = f'{self.year}-{self.year+1}'

        # Initialize filing object
        self.filing = Filing(self.season)

        ppr_coeff = 0.5 if kwargs.get('hppr', False) else 1.0        
        self.raw: pd.DataFrame = (self.filing.combined()
                                  .assign(fpts=lambda df: 0.04*df.pass_yds + 4.0*df.pass_td - 1.0*df.pass_int + 0.1*df.rush_yds + 6.0*df.rush_td + ppr_coeff*df.rec + 0.1*df.rec_yds + 6.0*df.rec_td - 2.0*df.fumbles_lost)
                                 )
        self.positions: pd.DataFrame = self.filing.positions()



In [5]:
cleaning = Cleaning(year=2022)

In [6]:
df = cleaning.raw

In [23]:
df[df.isna().any(axis=1)]

,name,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_long,pass_rating,rush_att,rush_yds,rush_td,rush_long,targets,rec,rec_yds,rec_td,rec_long,fumbles,fumbles_lost,opp,week,pos,fpts
206,Russell Wilson,DEN,17,25,237,2,0,3,23,55,124.9,4,29,1,13,1,1,1,0,1,0,0,NaN,NaN,NaN,27.48
207,Javonte Williams,DEN,0,0,0,0,0,0,0,0,0.0,10,28,0,13,1,1,-1,0,-1,0,0,NaN,NaN,NaN,3.70
208,Mike Boone,DEN,0,0,0,0,0,0,0,0,0.0,3,20,0,13,3,1,9,0,9,0,0,NaN,NaN,NaN,3.90
209,Melvin Gordon,DEN,0,0,0,0,0,0,0,0,0.0,3,8,0,7,1,0,0,0,0,1,1,NaN,NaN,NaN,-1.20
210,KJ Hamler,DEN,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,55,0,55,0,0,NaN,NaN,NaN,6.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5637,Davante Adams,LVR,0,0,0,0,0,0,0,0,0.0,0,0,0,0,9,8,95,0,16,0,0,NaN,NaN,NaN,17.50
5638,Hunter Renfrow,LVR,0,0,0,0,0,0,0,0,0.0,0,0,0,0,3,3,55,0,27,0,0,NaN,NaN,NaN,8.50
5639,Mack Hollins,LVR,0,0,0,0,0,0,0,0,0.0,0,0,0,0,3,2,44,1,26,0,0,NaN,NaN,NaN,12.40
5640,Foster Moreau,LVR,0,0,0,0,0,0,0,0,0.0,0,0,0,0,5,3,28,0,17,0,0,NaN,NaN,NaN,5.80


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5685 entries, 0 to 5684
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             5685 non-null   object 
 1   team             5685 non-null   object 
 2   pass_cmp         5685 non-null   int64  
 3   pass_att         5685 non-null   int64  
 4   pass_yds         5685 non-null   int64  
 5   pass_td          5685 non-null   int64  
 6   pass_int         5685 non-null   int64  
 7   pass_sacked      5685 non-null   int64  
 8   pass_sacked_yds  5685 non-null   int64  
 9   pass_long        5685 non-null   int64  
 10  pass_rating      5685 non-null   float64
 11  rush_att         5685 non-null   int64  
 12  rush_yds         5685 non-null   int64  
 13  rush_td          5685 non-null   int64  
 14  rush_long        5685 non-null   int64  
 15  targets          5685 non-null   int64  
 16  rec              5685 non-null   int64  
 17  rec_yds       

In [8]:
import numpy as np
import scipy.stats as stats

import itertools

In [10]:

team_cores = {
    'MIN': [
        'Justin Jefferson',
        'Dalvin Cook',
        'TJ Hockenson',
        'Kirk Cousins',
        'Adam Thielen'
    ],

    'PHI': [
        'Jalen Hurts',
        'DeVonta Smith',
        'AJ Brown',
        'Miles Sanders',
        'Quez Watkins'
    ]
}
team_dfs = {
    team: df.loc[(df['team'] == team) & (df['name'].isin(core)) & (df['fpts'] > 0.0)] 
    for team, core in team_cores.items()
}

In [11]:
team_core_together_weeks = {
    team: tuple(team_df
                .groupby('week')
                ['week']
                .agg(['count'])
                .pipe(lambda df_: df_.loc[df_['count'] == len(team_cores[team])])
                .index
               )
    for team, team_df in team_dfs.items()
}

In [12]:
team_core_fpts = {
    team: {
        name: [ (team_df.loc[(team_df['name'] == name) & (team_df['week'] == week ), 'fpts'].item()) for week in team_core_together_weeks[team] ]
        for name in team_cores[team]
    }

    for team, team_df in team_dfs.items()
}

In [19]:
team_combo_corrs = {
    team: dict(sorted({
        combo: stats.pearsonr(core_fpts[combo[0]], core_fpts[combo[1]])[0]
        for combo in itertools.combinations(team_cores[team], 2)
    }.items(), key=lambda item: item[1], reverse=True))
    for team, core_fpts in team_core_fpts.items()
}

In [20]:
team_combo_corrs['MIN']

{('Justin Jefferson', 'Kirk Cousins'): 0.7891543411540461,
 ('Justin Jefferson', 'Dalvin Cook'): 0.6047931286297824,
 ('Kirk Cousins', 'Adam Thielen'): 0.5490438704384552,
 ('Justin Jefferson', 'Adam Thielen'): 0.5294422443085328,
 ('Dalvin Cook', 'Kirk Cousins'): 0.5003357817949433,
 ('Justin Jefferson', 'TJ Hockenson'): 0.3869530406397159,
 ('TJ Hockenson', 'Kirk Cousins'): 0.28799418139025634,
 ('Dalvin Cook', 'Adam Thielen'): 0.1106548495986295,
 ('Dalvin Cook', 'TJ Hockenson'): -0.07555040805200897,
 ('TJ Hockenson', 'Adam Thielen'): -0.24850056241652038}

In [21]:
team_combo_corrs['PHI']

{('Jalen Hurts', 'Quez Watkins'): 0.5437401373083812,
 ('Jalen Hurts', 'DeVonta Smith'): 0.4570435146038356,
 ('Jalen Hurts', 'AJ Brown'): 0.3547797427501504,
 ('DeVonta Smith', 'AJ Brown'): 0.3391922304918648,
 ('Jalen Hurts', 'Miles Sanders'): 0.22478526110797653,
 ('Miles Sanders', 'Quez Watkins'): 0.1742267185422337,
 ('AJ Brown', 'Miles Sanders'): 0.050397553140670195,
 ('DeVonta Smith', 'Quez Watkins'): -0.0738482603069086,
 ('AJ Brown', 'Quez Watkins'): -0.25276317177977514,
 ('DeVonta Smith', 'Miles Sanders'): -0.2706424926800276}

In [62]:
min_core = [
    
]

min_core_df = min.loc[(min['name'].isin(min_core))] # & (buf['fpts'] > 0.0)]
# min_core_df

In [63]:
min_together_weeks = tuple(min_core_df
                           .groupby('week')
                           ['week']
                           .agg(['count'])
                           .pipe(lambda df_: df_.loc[df_['count'] == len(min_core)])
                           .index
                          )

In [64]:
min_core_values = {
    name: [(min_core_df
            .loc[(min_core_df['name'] == name) & (min_core_df['week'] == week)]
            ['fpts']
            .item()
           )
           for week in min_together_weeks
          ]
    for name in min_core
}

In [153]:
min_combo_corrs = {
    combo: stats.pearsonr(min_core_values[combo[0]], min_core_values[combo[1]])[0]
    for combo in itertools.combinations(min_core, 2)
}


dict(sorted(min_combo_corrs.items(), key=lambda item: item[1], reverse=True))

{('Justin Jefferson', 'Kirk Cousins'): 0.7891543411540461,
 ('Justin Jefferson', 'Dalvin Cook'): 0.6047931286297824,
 ('Kirk Cousins', 'Adam Thielen'): 0.5490438704384552,
 ('Justin Jefferson', 'Adam Thielen'): 0.5294422443085328,
 ('Dalvin Cook', 'Kirk Cousins'): 0.5003357817949433,
 ('Justin Jefferson', 'TJ Hockenson'): 0.3869530406397159,
 ('TJ Hockenson', 'Kirk Cousins'): 0.28799418139025634,
 ('Dalvin Cook', 'Adam Thielen'): 0.1106548495986295,
 ('Dalvin Cook', 'TJ Hockenson'): -0.07555040805200897,
 ('TJ Hockenson', 'Adam Thielen'): -0.24850056241652038}

In [ ]:
stats.pearsonr(
    min_core_values['Justin Jefferson'],
    min_core_values['Dalvin Cook']
)[0]

In [136]:
buf_core = [
    'Josh Allen',
    'Stefon Diggs',
    'Dawson Knox',
    'Gabriel Davis',
    'James Cook',
    'Isaiah McKenzie',
    'Devin Singletary',
    'Khalil Shakir'
]

buf_core_df = buf.loc[(buf['name'].isin(buf_core))] # & (buf['fpts'] > 0.0)]
(buf_core_df
 .groupby('name')
 ['fpts']
 .agg([np.mean, np.median, np.std])
 .sort_values('median', ascending=False)
)

,mean,median,std
name,,,
Josh Allen,25.345000,25.44,6.464733
Stefon Diggs,19.787500,21.95,10.081923
Devin Singletary,11.118750,9.65,6.054994
Gabriel Davis,11.440000,9.50,7.798700
Dawson Knox,9.046667,9.00,5.556061
James Cook,6.606250,6.20,5.556674
Isaiah McKenzie,7.986667,5.80,6.564174
Khalil Shakir,3.210000,1.80,4.907013


In [130]:
buf_together_weeks = tuple(buf_core_df
                           .groupby('week')
                           ['week']
                           .agg(['count'])
                           .pipe(lambda df_: df_.loc[df_['count'] == len(buf_core)])
                           .index
                          )

buf_core_values = {
    name: [(buf_core_df
            .loc[(buf_core_df['name'] == name) & (buf_core_df['week'] == week)]
            ['fpts']
            .item()
           )
           for week in buf_together_weeks
          ]
    for name in buf_core
}

In [152]:
buf_combo_corrs = {
    combo: stats.pearsonr(buf_core_values[combo[0]], buf_core_values[combo[1]])[0]
    for combo in itertools.combinations(buf_core, 2)
    # if 'Josh Allen' in combo
    # if 'Stefon Diggs' in combo
}

dict(sorted(buf_combo_corrs.items(), key=lambda item: item[1], reverse=True))

{('Josh Allen', 'Dawson Knox'): 0.5832355663541106,
 ('Gabriel Davis', 'Devin Singletary'): 0.5605418929396544,
 ('Josh Allen', 'Khalil Shakir'): 0.5022843324341338,
 ('Stefon Diggs', 'Gabriel Davis'): 0.4792517800111933,
 ('James Cook', 'Devin Singletary'): 0.2670018086428258,
 ('Josh Allen', 'Devin Singletary'): 0.2515638864207145,
 ('Dawson Knox', 'Gabriel Davis'): 0.1867769611368109,
 ('Josh Allen', 'Gabriel Davis'): 0.16919168331338916,
 ('Isaiah McKenzie', 'Khalil Shakir'): 0.16260636480438118,
 ('Stefon Diggs', 'Khalil Shakir'): 0.1326750685342753,
 ('Dawson Knox', 'Devin Singletary'): 0.10159695307110637,
 ('Stefon Diggs', 'Isaiah McKenzie'): 0.08742814635421764,
 ('Josh Allen', 'Stefon Diggs'): 0.03947466383993915,
 ('Gabriel Davis', 'James Cook'): -0.0606348552239777,
 ('Devin Singletary', 'Khalil Shakir'): -0.07348263443374012,
 ('Stefon Diggs', 'Devin Singletary'): -0.11184508529576606,
 ('Gabriel Davis', 'Khalil Shakir'): -0.13448640665369999,
 ('Isaiah McKenzie', 'Devin S

In [ ]:
stats.pearsonr(
    buf_core_values['Josh Allen'],
    buf_core_values['Stefon Diggs']
)[0]

In [ ]:
df.info()

In [ ]:
teams = [
    'NYJ',
    'BUF'
]

agg_stats = ['rush_td', 'rush_yds', 'rec_td', 'rec_yds', 'rec', 'fpts']

In [ ]:
agg_df = (df
          .groupby('opp')
          [agg_stats]
          .agg(['sum'])
          .reset_index()
          .set_axis(['opp'] + agg_stats, axis=1)
          .pipe(lambda df_: df_.loc[df_['opp'].isin(teams)])
          .set_index('opp')
          .assign(
              total_yds=lambda df_: df_.rush_yds + df_.rec_yds,
              total_td=lambda df_: df_.rush_td + df_.rec_td 
          )
         )

In [ ]:
agg_df.sort_values([
    'fpts',
    # 'total_td',
    # 'total_yds',
    'rec',
    # 'rush_td',
    'rush_yds',
], ascending=False)